In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    "../datasets/local/vis_papers/IEEE VIS papers 1990-2022 - Main dataset.csv"
)

In [22]:
abstracts = df["Abstract"]
abstracts_non_null = abstracts.dropna().reset_index(drop=True)

In [4]:
from textprofilerbackend.transform import get_word_tokens_batch, get_byte_encoding_batch
from textprofilerbackend.textclean import get_textcol_metadata_embeddings

/Users/wepperso/miniconda3/envs/textprofiler/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_names = [
    "all-mpnet-base-v2",  # best performing on leaderboard
    # "all-MiniLM-L6-v2",  # smaller and faster
]

In [26]:
meta_data_dict = get_textcol_metadata_embeddings(abstracts_non_null, model_names)

Created embedding of shape torch.Size([3549, 768]) with all-mpnet-base-v2


In [27]:
meta_data_dict["heuristic_metadata"]

,Abstract_text_length,Abstract_num_words,Abstract_max_word_length,Abstract_avg_word_length,Abstract_perc_special_chars
0,2109,286,22,6.377622,0.016121
1,1153,167,14,5.910180,0.019948
2,1543,220,16,6.018182,0.017498
3,1472,223,15,5.605381,0.025136
4,1879,272,17,5.911765,0.027674
...,...,...,...,...,...
3544,758,108,28,6.027778,0.026385
3545,786,111,29,6.090090,0.036896
3546,898,138,25,5.514493,0.026726
3547,879,116,26,6.586207,0.018203


In [28]:
meta_data_dict["model_embeddings"]["all-mpnet-base-v2"]

tensor([[-0.0186,  0.0270, -0.0430,  ...,  0.0387, -0.0176, -0.0138],
        [-0.0256,  0.0685, -0.0391,  ..., -0.0113, -0.0013,  0.0081],
        [-0.0303, -0.0032, -0.0174,  ...,  0.0170, -0.0464, -0.0170],
        ...,
        [-0.0179, -0.0226, -0.0176,  ..., -0.0003,  0.0411,  0.0018],
        [-0.0503,  0.0167, -0.0140,  ...,  0.0455, -0.0256, -0.0077],
        [-0.0025,  0.0178, -0.0276,  ...,  0.0423, -0.0274,  0.0337]])

In [29]:
words = get_word_tokens_batch(abstracts_non_null)

In [30]:
tokens = get_byte_encoding_batch(abstracts_non_null)

In [33]:
df_non_null = df[df.Abstract.notnull()]

In [38]:
df_non_null = df_non_null.reset_index(drop=True)

In [41]:
df_with_metadat = df_non_null.join(meta_data_dict["heuristic_metadata"])

In [44]:
df_with_metadat = df_with_metadat.reset_index().rename(columns={"index": "id"})

In [46]:
df_with_metadat.to_parquet("../datasets/local/vis_papers/processed/vis_papers.parquet")

In [53]:
word_long_format = pd.DataFrame(
    [(word, i) for i, doc in enumerate(words) for word in doc], columns=["word", "id"]
)

In [54]:
token_long_format = pd.DataFrame(
    [(token, i) for i, doc in enumerate(tokens) for token in doc],
    columns=["token", "id"],
)

In [58]:
token_long_format.to_parquet(
    "../datasets/local/vis_papers/processed/vis_papers_tokens.parquet"
)
word_long_format.to_parquet(
    "../datasets/local/vis_papers/processed/vis_papers_words.parquet"
)

In [60]:
meta_data_dict["model_embeddings"]["all-mpnet-base-v2"].shape

torch.Size([3549, 768])

In [63]:
import torch

In [64]:
torch.save(
    meta_data_dict["model_embeddings"]["all-mpnet-base-v2"],
    "../datasets/local/vis_papers/processed/vis_papers_embeddings.pt",
)